In [2]:
from math import floor, log
from collections import defaultdict
import random
import tqdm
import numpy as np
from numpy.random import poisson, binomial, uniform, exponential, multinomial, normal, multivariate_normal
from scipy.stats import norm
from scipy.stats import kstest, norm
import scipy.linalg as la
def multi(dist):
    v = multinomial(1, dist)
    v = np.where(v == 1)[0][0]
    return v

def markov_start(start, P, length):
    '''
    Runs Markov Chain with a start value, generates {length} items. 
    '''
    cur = start
    seq = []
    for i in range(length):
        distribution = P[cur].flatten()
        v = multinomial(1, distribution)
        v = np.where(v == 1)[0][0]
        seq.append(v)
        cur = v
    return seq

def markov_start_absorb(start, P, absorb_set):
    '''
    Runs Markov Chain with a start value, generates {length} items. 
    '''
    cur = start
    seq = []
    while True:
        distribution = P[cur].flatten()
        v = multi(distribution)
        seq.append(v)
        cur = v
        if cur in absorb_set:
            break
    return seq

def markov_rdm(dist, P, length):
    '''
    Runs Markov Chain with a starting distribution, generates {length} + 1 items. 
    '''
    cur = np.nonzero(multinomial(1, dist))[0]
    seq = [cur]
    for i in range(length):
        distribution = P[cur].flatten()
        v = np.nonzero(multinomial(1, distribution))[0]
        seq.append(v)
        cur = v
    return seq

In [3]:
mat = np.array([[0.8,0.15,0.05],[0.05,0.9,0.05],[0.05,0.1,0.85]]) 
from scipy.linalg import lu, solve
def stationary(mat):
    pl, u = lu(mat.T - np.eye(len(mat)), permute_l = True)
    u[-1,:] = 1
    targets = np.zeros(len(mat))
    targets[-1] = 1
    return solve(u, targets)
stationary(mat)

array([0.2 , 0.55, 0.25])

In [4]:
stationary(np.array([[0, 1, 0, 0], [0.6, 0, 0.4, 0], [0, 0.75, 0, 0.25], [0, 0, 0.75, 0.25]]))

array([0.25961538, 0.43269231, 0.23076923, 0.07692308])

In [4]:
p = 0.6
q = 1 - p
p2,p3,p4,p5=p**2,p**3,p**4,p**5
q2,q3,q4,q5=q**2,q**3,q**4,q**5
t=p/q
P2 = np.zeros((6, 6))
for i in range(5):
    P2[i,i + 1] = p
    P2[i + 1,i] = q
P2[0,0] = q
P2[5, 5] = p
print("P2")
print(P2)
print("stationary: ")
print(stationary(P2), np.sum(stationary(P2).reshape((1,-1)) @ P2 - stationary(P2)) <= 1e-6, abs(np.sum(stationary(P2)) - 1) <= 1e-6)

P2
[[0.4 0.6 0.  0.  0.  0. ]
 [0.4 0.  0.6 0.  0.  0. ]
 [0.  0.4 0.  0.6 0.  0. ]
 [0.  0.  0.4 0.  0.6 0. ]
 [0.  0.  0.  0.4 0.  0.6]
 [0.  0.  0.  0.  0.4 0.6]]
stationary: 
[0.0481203  0.07218045 0.10827068 0.16240602 0.24360902 0.36541353] True True


In [8]:
matlab = q**5/((p*q - 1)*(3*p*q - 1))
print(matlab)

0.04812030075187971


In [5]:
def q2i():
    pi0 = 1/np.sum([(p/q) ** i for i in range(6)])
    pi1_5 = [(p/q) ** i * pi0 for i in range(1, 6)]
    print([pi0] + pi1_5)
q2i()

[0.048120300751879716, 0.07218045112781957, 0.10827067669172934, 0.16240601503759397, 0.24360902255639094, 0.36541353383458636]


In [96]:
ct = 15000
def simulate():
    sums = [0]*6
    for i in tqdm.tqdm(range(ct)):
        for j in range(6):
            sums[j] += len(markov_start_absorb(j, P2, {j}))
    sums2 = np.array(sums) / ct
    print(sums2, 1/sums2)
simulate()

def simulate2_to_zero():
    sums = [0]*6
    for i in tqdm.tqdm(range(ct)):
        for j in range(6):
            sums[j] += len(markov_start_absorb(j, P2, {0}))
    sums2 = np.array(sums) / ct
    print(sums2, 1/sums2)
simulate2_to_zero()

100%|██████████| 15000/15000 [00:13<00:00, 1145.30it/s]


[21.20406667 13.7484      9.23633333  6.18606667  4.15333333  2.75993333] [0.04716076 0.07273574 0.10826807 0.16165361 0.24077047 0.36232759]


100%|██████████| 15000/15000 [01:11<00:00, 209.11it/s]

[21.05866667 33.167      54.90126667 65.0736     71.13293333 73.61746667] [0.04748639 0.03015045 0.01821452 0.01536721 0.01405819 0.01358373]


In [15]:
matlab1 = -(p**4 - 2*p**3 + 4*p**2 - 3*p + 1)/(p - 1)**5
matlab2 = -(p**2*q**2 - 3*p*q + 1)/(p**3*q**2 - 3*p**2*q**2 - 3*p**2*q + 4*p*q + p - 1)
matlab3 = -(p**2*q**2 - 3*p*q + 1)/(p**3*q**2 - 3*p**2*q**2 - 3*p**2*q + 4*p*q + p - 1)
print(1/matlab1, 1/matlab2, 1/(1 + p * matlab3))

0.030331753554502364 0.030331753554502777 0.04812030075188032


In [9]:
from scipy.linalg import inv
def verify(h2):
    h1 = 1 + p * h2
    ans = 1 + p * h1
    return 1/ans

def verifyh1(h1):
    ans = 1 + p * h1
    return 1/ans
def q23():
    Z = np.zeros((5, 5))
    for i in range(4):
        Z[i,i+1] = p
        Z[i+1,i] = q
    Z[4,4] = p
    print(Z)
    solution = inv(np.eye(5) - Z) @ np.ones((5, 1)).flatten()
    print(solution)
    print(1/(1 + p * solution[0]))
    print(verify(solution[1]))
    # solution = (inv(np.eye(6) - P2) @ np.ones((6, 1))).flatten()
    # print(solution)

    a,b,c,d,e = 1-2*q*p,-p2, q2, -q2, -1-q-t
    thing1 = (2*d-e*b)/(a*d-c*b)

    thing2 = (-2 * q2 - p2 - p2 * q - p3 / q) / ((-q2) * (-p2 + 1 - 2 * q * p))
    thing3 = (2*q3 + p2*q + p2 * q2 + p3) / q5

    thing4 = (q5 + 2*q3*p + p3*q + p3*q2+p4)/q5
    
    thing5 = 1 + p*thing4
    thing6 = (q5 + q5*p + 2*q3*p2+p4*q+p4*q2+p5) / q5
    h2s = [thing1, thing2, thing3]
    h1s = [thing4]
    sols = [thing5, thing6]
    print([verify(w) for w in h2s])
    print([verifyh1(w) for w in h1s])
    print([1/w for w in sols])
q23()

[[0.  0.6 0.  0.  0. ]
 [0.4 0.  0.6 0.  0. ]
 [0.  0.4 0.  0.6 0. ]
 [0.  0.  0.4 0.  0.6]
 [0.  0.  0.  0.4 0.6]]
[32.96875 53.28125 65.15625 71.40625 73.90625]
0.048120300751879834
0.04812030075187982
[0.04812030075187972, 0.04812030075187972, 0.04812030075187972]
[0.04812030075187971]
[0.04812030075187971, 0.048120300751879716]


In [38]:
def q2():
    p = 0.75
    q = 1 - p
    mat = np.zeros((6, 6))
    for i in range(5):
        mat[i,i + 1] = p
        mat[i + 1,i] = q
    mat[0,0] = q
    mat[5, 5] = p
    print("mat: ")
    print(mat)
    print("mat.T: ")
    print(mat.T)
    pi = la.null_space(mat.T - np.eye(6)).flatten()
    # print(pi.reshape((1, 6)) @ mat)
    print("pi: (null space of P.T)", np.around(pi, decimals=3))

    t = p / q
    d = pi[1] - pi[0]
    # print("p1 - p0", null[1] - null[0])
    # print("(p/q) (p1 - p0)", t * (null[1] - null[0]))
    # print("p2 - p1", null[2] - null[1])

    emp_inc = [pi[i + 1] - pi[i] for i in range(5)]
    ana_inc = [t ** i *  (pi[1] - pi[0]) for i in range(5)]

    true_x_min_0 = [pi[i] - pi[0] for i in range(6)]
    clos_x_min_0 = list([pi[0]]) + [np.sum([t ** j * d for j in range(i + 1)]).item() for i in range(5)]
    clos2_x_min_0 = list([pi[0]]) + [d * ((1 - t ** (i + 1)) / (1 - t)) for i in range(5)]
    print("x[i] - x[0],            ", np.around(true_x_min_0, decimals = 3))
    print("closedform x[i] - x[0], ", np.around(clos_x_min_0, decimals = 3))
    print("closedform2 x[i] - x[0],", np.around(clos2_x_min_0, decimals = 3))

    

    # print("emp_inc, ", emp_inc)
    # print("ana_inc", ana_inc)

    clos2_orig = list([pi[0]]) + [d * ((1 - t ** (i + 1)) / (1 - t)) + pi[0] for i in range(5)]
    print("should be pi: ", np.around(clos2_orig, decimals=3))
    
    emp_2_5 = np.sum(pi[2:6])
    ana_2_5 = np.sum([d * (1 - t ** (i + 1)) / (1 - t) + pi[0] for i in range(1, 5)])

    emp_sum_n_1_4 = np.sum([(1 - t ** (i + 1)) / (1 - t)  for i in range(1, 5)])
    ana_sum_n_1_4 = 4/(1-t) - t ** 2 * (1 - t ** 4) / ((1 - t) ** 2)
    # ana2_2_5 = -d * t * (1 - t ** 4) / (1 - t) + 4 * pi[0] + 4 * d / (1 - t)
    print("sum n 2-5:    ", np.around(emp_sum_n_1_4, decimals = 3))
    print("closed n 2-5: ", np.around(ana_sum_n_1_4, decimals = 3))

    print("true 2-5:   ", np.around(emp_2_5, decimals = 3))
    print("closed 2-5: ", np.around(ana_2_5, decimals = 3))
    # print("closed2 2-5:", np.around(ana2_2_5, decimals = 3))

    full_sum = np.sum(pi)
    emp_sum = pi[0] + pi[1] + 4 * pi[0] + d * 4 / (1 - t) - d * t ** 2 * (1 - t ** 4) / ((1 - t) ** 2)

    print("full_sum: ", full_sum)
    print("norm_full_sum: ", pi / full_sum, (pi / full_sum).reshape((1, 6)) @ mat)

    print("emp_sum:  ", emp_sum)
    thing = 5 + (1-q)/q + 4 * (1-2 * q) / (1 - t) / q - (1 - 2 * q) * t ** 2 * (1 - t ** 4) / (1 - t) / (1 - t) / q
    thing2 = 1 + p/q + p ** 2 / q ** 2 * (1 - t ** 4) / (1 - t)
    thing3 = q ** 5 / (p ** 5 + p**4 * q + p**3 * q**2 + p **2 * q**3 + p * q**4 + q ** 5)
    print(thing, thing2, 1/thing3)

    pi_z = (1-t)/(1-t ** 6)
    pi_1_5 = [pi_z * (p/q) ** i for i in range(1, 6)]
    pi2 = [pi_z] + pi_1_5

    print("pi: (null space of P.T)", np.around(pi/np.sum(pi), decimals=3))
    print("pi2 empirical: ", np.around(pi2, decimals=3))
    ans = 1 / thing
    print(ans)
    
def q2_cut():
    p = 0.65
    q = 1 - p
    mat = np.zeros((6, 6))
    for i in range(5):
        mat[i,i + 1] = p
        mat[i + 1,i] = q
    mat[0,0] = q
    mat[5, 5] = p
    assert(all(np.sum(mat, axis=1) == 1))
    print(f"p: {p}, q: {q}")
    print(mat)
    for row in mat:
        for v in row:
            print(np.around(v, decimals=2), end=" ")
        print()
    pi = la.null_space(mat.T - np.eye(6)).flatten()
    # print(pi.reshape((1, 6)) @ mat)
    print("pi: (null space of P.T)", np.around(pi/np.sum(pi), decimals=7))

q2()

mat: 
[[0.25 0.75 0.   0.   0.   0.  ]
 [0.25 0.   0.75 0.   0.   0.  ]
 [0.   0.25 0.   0.75 0.   0.  ]
 [0.   0.   0.25 0.   0.75 0.  ]
 [0.   0.   0.   0.25 0.   0.75]
 [0.   0.   0.   0.   0.25 0.75]]
mat.T: 
[[0.25 0.25 0.   0.   0.   0.  ]
 [0.75 0.   0.25 0.   0.   0.  ]
 [0.   0.75 0.   0.25 0.   0.  ]
 [0.   0.   0.75 0.   0.25 0.  ]
 [0.   0.   0.   0.75 0.   0.25]
 [0.   0.   0.   0.   0.75 0.75]]
pi: (null space of P.T) [0.004 0.012 0.035 0.105 0.314 0.943]
x[i] - x[0],             [0.    0.008 0.031 0.101 0.31  0.939]
closedform x[i] - x[0],  [0.004 0.008 0.031 0.101 0.31  0.939]
closedform2 x[i] - x[0], [0.004 0.008 0.031 0.101 0.31  0.939]
should be pi:  [0.004 0.012 0.035 0.105 0.314 0.943]
sum n 2-5:     178.0
closed n 2-5:  178.0
true 2-5:    1.397
closed 2-5:  1.397
full_sum:  1.4122749547964117
norm_full_sum:  [0.00274725 0.00824176 0.02472527 0.07417582 0.22252747 0.66758242] [[0.00274725 0.00824176 0.02472527 0.07417582 0.22252747 0.66758242]]
emp_sum:   1.4122749

In [32]:
def q2iii():
    P = np.zeros((5, 5))
    p = 0.65
    q = 1- p
    for i in range(4):
        P[i,i+1] = p
        P[i+1,i] = q
    P[4,4] = p
    # print("P: ")
    # print(P)
    # print("I - P")
    # print(np.eye(5) - P)
    # print("inv(I - P)")
    # print(np.linalg.inv(np.eye(5) - P))
    # print("(I - P) @ 1 vector")
    # print(np.linalg.inv(np.eye(5) - P) @ np.ones((5, 1)))
    sol = np.linalg.inv(np.eye(5) - P) @ np.ones((5, 1))
    print(sol)
    print(1/ (1 + 0.5*sol[0]))
    val = -(p**2*q**2 - 3*p*q + 1)/(p**3*q**2 - 3*p**2*q**2 - 3*p**2*q + 4*p*q + p - 1)

    ans = 1 + 0.5 * sol[0][0]
    ans2 = 1 + 0.5 * val

    a, b, c, d, e = 1 - 2 * q * p, -p * p, q * q, - q * q, -1 - q - p/q
    thing = (2*d - e * b) / (a * d - c * b)
    thing2 = (2 * (-(q ** 2)) - (-1 - q - p/q) * (-(p ** 2))) / ((1 - 2 * q * p) * (-q ** 2) + (q ** 2 * p ** 2))

    q2, q3, q4, q5 = q**2, q**3, q**4, q**5
    p2, p3, p4, p5 = p**2, p**3, p**4, p**5
    thing3 = (-2 * q2 - p2 - q * p2 - p * p2 / q) / (-q2 + 2 * q * q2 * p + q2 * p2)
    thing4 = (-2 * q3 - q * p2 - q2 * p2 - p3) / (q3 * (-1 + 2*p*q + p2))
    thing5 = (2* q3 + q * p2 + q2 * p2 + p3) / q5
    h1 = 1 + p * thing
    print("things: ", thing, thing2, thing3, thing4, thing5)


    t0 = p/q+p2/q2+p3/q3+p4/q4+p5/q5
    t1 = 0.5 * (1 + p * thing)
    t2 = 0.5 * ((q5 + 2*q3*p + q*p3 + q2*p3 + p4)/q5)
    ts = [t0, t1, t2]
    print("1/t1s: ", [round(1/(1 + n),5) for n in ts])
    print("h1: ", h1)
    ans3 = 1 + 0.5 * h1
    print("1/ans3: ", 1/ans3)

    # print(sol)
    print(1/ans, 1/ans2)
    # print((1-p/q)/(1-(p/q) ** 6))

q2iii()


[[ 70.30522996]
 [106.62343071]
 [124.64092342]
 [132.80418873]
 [135.66133159]]
[0.02766052]
things:  106.62343071339319 106.62343071339319 106.6234307133932 106.62343071339326 106.62343071339328
1/t1s:  [0.02141, 0.02766, 0.02766]
h1:  70.30522996370557
1/ans3:  0.027660516410831174
0.027660516410831153 0.0276605164108312


In [52]:
def q3():
    P = np.array([[0.8, 0.15, 0.05], [0.05, 0.9, 0.05], [0.05, 0.1, 0.85]])
    print(stationary(P))
q3()

[0.2  0.55 0.25]


In [51]:
from scipy.linalg import lu, solve
def q5():
    P = np.zeros((6, 6))
    for i in range(5):
        P[i,i+1] = 0.5
        P[i+1,i] = 0.5
    P[0,0] = 0.5
    P[5,5] = 0.5
    print(stationary(P))
    # print("Original: ")
    # print(P)
    # pl, u = lu(P - np.eye(6), permute_l = True)
    # print("Row reduced: ")
    # print(u)
    # u[5,:] = 1
    # print("added row of ones: ")
    # print(u)
    # targets = np.zeros(6)
    # targets[5] = 1
    # print("final: ")
    # print(solve(u, targets))
q5()

[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
